 ## Required Libraries

In [3]:
!pip install pandas matplotlib seaborn wordcloud
# For sentiment analysis, we'll use a library called vaderSentiment
!pip install vaderSentiment

## Data Cleaning and Preprocessing 🧹

In [10]:
import pandas as pd
import os

# --- 1. Load the Datasets and Add a 'region' Column ---
# The path you got from kagglehub might be a zip file.
# First, ensure the files are extracted into a folder.
# Let's assume the path is '.../youtube-new/' where the CSVs are located.
# Replace 'path/to/your/files' with the actual path printed in your last step.

data_path = r'C:\Users\rrp28\YouTube Trending Video Analytics' # IMPORTANT: Change this to your actual path

files_to_load = {
    'US': 'USvideos.csv',
    'GB': 'GBvideos.csv',
    'IN': 'INvideos.csv',
    'CA': 'CAvideos.csv'
}

all_dfs = []
for region, filename in files_to_load.items():
    # Construct the full file path
    file_path = os.path.join(data_path, filename)
    try:
        # Load the CSV file
        df = pd.read_csv(file_path, encoding='latin1') # 'latin1' encoding helps with special characters
        # Add the region column
        df['region'] = region
        all_dfs.append(df)
    except FileNotFoundError:
        print(f"Warning: {filename} not found at {file_path}. Skipping.")

# Concatenate all DataFrames into one
videos_df = pd.concat(all_dfs, ignore_index=True)


# --- 2. Load the Category JSON and Map Category Names ---

# Load the JSON file (we'll use the US one as it's generally comprehensive)
json_path = os.path.join(data_path, 'US_category_id.json')
category_df = pd.read_json(json_path)

# Create a dictionary to map category_id to category name
category_map = {}
for item in category_df['items']:
    category_map[int(item['id'])] = item['snippet']['title']

# Map the category names to our main DataFrame
videos_df['category_name'] = videos_df['category_id'].map(category_map)


# --- 3. Initial Inspection ---

print("--- First 5 Rows of the Combined Dataset ---")
print(videos_df.head())

print("\n\n--- Dataset Information ---")
videos_df.info()

print("\n\n--- Missing Values ---")
print(videos_df.isnull().sum())

--- First 5 Rows of the Combined Dataset ---
      video_id trending_date  \
0  2kyS6SvSYSE      17.14.11   
1  1ZAPwfrtAFY      17.14.11   
2  5qpjK5DgCt4      17.14.11   
3  puqaWrEC7tY      17.14.11   
4  d380meD0W0M      17.14.11   

                                               title          channel_title  \
0                 WE WANT TO TALK ABOUT OUR MARRIAGE           CaseyNeistat   
1  The Trump Presidency: Last Week Tonight with J...        LastWeekTonight   
2  Racist Superman | Rudy Mancuso, King Bach & Le...           Rudy Mancuso   
3                   Nickelback Lyrics: Real or Fake?  Good Mythical Morning   
4                           I Dare You: GOING BALD!?               nigahiga   

   category_id              publish_time  \
0           22  2017-11-13T17:13:01.000Z   
1           24  2017-11-13T07:30:00.000Z   
2           23  2017-11-12T19:05:24.000Z   
3           24  2017-11-13T11:00:04.000Z   
4           24  2017-11-12T18:01:41.000Z   

                      

In [11]:
# --- 1. Fix Date and Time Columns ---

# The 'trending_date' is in YY.DD.MM format. We specify this format for pandas to understand it correctly.
videos_df['trending_date'] = pd.to_datetime(videos_df['trending_date'], format='%y.%d.%m')

# The 'publish_time' is in ISO 8601 format, which pandas can often infer automatically.
# We'll convert it and then extract just the date part for easier comparison later.
videos_df['publish_time'] = pd.to_datetime(videos_df['publish_time'])


# --- 2. Handle Missing Descriptions ---

# We'll fill the NaN (Not a Number) values in the description with an empty string.
videos_df['description'].fillna('', inplace=True)


# --- 3. Final Check ---

print("--- Data Types After Cleaning ---")
videos_df.info()

print("\n\n--- Missing Values After Cleaning ---")
print(videos_df.isnull().sum())

print("\n\n--- Sample of Cleaned Date Columns ---")
print(videos_df[['trending_date', 'publish_time']].head())

--- Data Types After Cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158098 entries, 0 to 158097
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   video_id                158098 non-null  object             
 1   trending_date           158098 non-null  datetime64[ns]     
 2   title                   158098 non-null  object             
 3   channel_title           158098 non-null  object             
 4   category_id             158098 non-null  int64              
 5   publish_time            158098 non-null  datetime64[ns, UTC]
 6   tags                    158098 non-null  object             
 7   views                   158098 non-null  int64              
 8   likes                   158098 non-null  int64              
 9   dislikes                158098 non-null  int64              
 10  comment_count           158098 non-null  int64            

C:\Users\rrp28\AppData\Local\Temp\ipykernel_13956\2869788280.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  videos_df['description'].fillna('', inplace=True)


## Exploratory Data Analysis (EDA) & SQL 📊

### Top 10 Trending Categories

In [12]:
import sqlite3

# --- 1. Create a database connection ---
# This will create a file named 'youtube_trends.db' in your folder
conn = sqlite3.connect('youtube_trends.db')

# --- 2. Save our DataFrame to an SQL table ---
# We'll name the table 'videos'. If it already exists, we'll replace it.
videos_df.to_sql('videos', conn, if_exists='replace', index=False)

print("DataFrame has been successfully saved to the 'videos' table in youtube_trends.db")


# --- 3. Write and Execute our First SQL Query ---
# Let's find the top 10 categories with the most trending videos across all regions.
query = """
SELECT
    category_name,
    COUNT(*) AS number_of_videos
FROM
    videos
GROUP BY
    category_name
ORDER BY
    number_of_videos DESC
LIMIT 10;
"""

# Use pandas to run the query and get the result as a DataFrame
top_categories = pd.read_sql_query(query, conn)


# --- 4. Close the connection and view results ---
conn.close()

print("\n\n--- Top 10 Trending Categories (All Regions) ---")
print(top_categories)

DataFrame has been successfully saved to the 'videos' table in youtube_trends.db


--- Top 10 Trending Categories (All Regions) ---
          category_name  number_of_videos
0         Entertainment             49251
1                 Music             27815
2       News & Politics             13112
3        People & Blogs             12865
4                Comedy             12487
5         Howto & Style              8926
6      Film & Animation              8640
7                Sports              7599
8  Science & Technology              4626
9             Education              4331


### Which Categories Get the Most Views?

In [13]:
import sqlite3
import pandas as pd

# --- 1. Connect to our database ---
conn = sqlite3.connect('youtube_trends.db')

# --- 2. Write and Execute the New SQL Query ---
# Query to rank categories by their average view count.
# We'll also get average likes and comments to be more thorough.
query = """
SELECT
    category_name,
    ROUND(AVG(views)) AS average_views,
    ROUND(AVG(likes)) AS average_likes,
    ROUND(AVG(comment_count)) AS average_comments
FROM
    videos
GROUP BY
    category_name
ORDER BY
    average_views DESC
LIMIT 10;
"""

# Run the query
performance_by_category = pd.read_sql_query(query, conn)

# --- 3. Close the connection and view results ---
conn.close()

print("\n\n--- Top 10 Categories by Average Views ---")
print(performance_by_category)



--- Top 10 Categories by Average Views ---
           category_name  average_views  average_likes  average_comments
0                  Music      8435177.0       214942.0           17557.0
1                 Movies      3007296.0        41616.0            2315.0
2       Film & Animation      2596421.0        53294.0            5712.0
3  Nonprofits & Activism      1975326.0       152543.0           48212.0
4          Entertainment      1628033.0        41790.0            5847.0
5                 Sports      1595446.0        34995.0            4162.0
6   Science & Technology      1519962.0        38611.0            7110.0
7                 Gaming      1345976.0        49215.0            7770.0
8                 Comedy      1255354.0        58451.0            5823.0
9         People & Blogs      1186460.0        35025.0            4895.0


## Sentiment Analysis

In [14]:
# Make sure you have it installed: pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd # Ensure pandas is imported

# We need to work with our original DataFrame again
# If you don't have videos_df in memory, you might need to reload it and clean it again.
# For now, we'll assume 'videos_df' is still available from our earlier steps.

# 1. Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# 2. Define a function to get the sentiment
def get_sentiment(title):
    # The 'compound' score is a single metric for the overall sentiment
    # Ranges from -1 (most negative) to +1 (most positive)
    compound_score = analyzer.polarity_scores(title)['compound']
    
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# 3. Apply the function to our 'title' column
# This will take a moment as it processes over 150,000 titles.
videos_df['sentiment'] = videos_df['title'].apply(get_sentiment)

# 4. Analyze the results
sentiment_counts = videos_df['sentiment'].value_counts()

print("--- Sentiment Analysis of Video Titles ---")
print(sentiment_counts)

# You can also see the percentage
print("\n--- Sentiment Distribution (%) ---")
print(videos_df['sentiment'].value_counts(normalize=True) * 100)

--- Sentiment Analysis of Video Titles ---
sentiment
Neutral     91769
Positive    37493
Negative    28836
Name: count, dtype: int64

--- Sentiment Distribution (%) ---
sentiment
Neutral     58.045643
Positive    23.715038
Negative    18.239320
Name: proportion, dtype: float64


## Time-Series Analysis (Trend Duration) ⏳

In [15]:
# We'll continue working with our 'videos_df' DataFrame

# 1. Calculate trending duration for each video
# We group by video_id and use transform('count') to create a new column
# that shows how many times each video_id appeared.
videos_df['trending_duration_days'] = videos_df.groupby('video_id')['video_id'].transform('count')

# 2. Calculate the overall average trending duration
# We'll look at the duration for unique videos only to get an accurate average.
# We first drop duplicates to avoid counting the same video multiple times.
average_duration = videos_df[['video_id', 'trending_duration_days']].drop_duplicates()['trending_duration_days'].mean()

print(f"--- Trend Duration Analysis ---")
print(f"The average trending duration for a video is: {average_duration:.2f} days")

# 3. Analyze the average duration by category
duration_by_category = videos_df.groupby('category_name')['trending_duration_days'].mean().round(2).sort_values(ascending=False)

print("\n\n--- Average Trending Duration by Category (in Days) ---")
print(duration_by_category.head(10))

--- Trend Duration Analysis ---
The average trending duration for a video is: 3.50 days


--- Average Trending Duration by Category (in Days) ---
category_name
Music                   23.21
Movies                  18.36
Film & Animation        16.86
Pets & Animals          15.86
Gaming                  12.20
Entertainment           11.75
Science & Technology    11.53
Howto & Style           11.41
Comedy                  10.53
People & Blogs          10.37
Name: trending_duration_days, dtype: float64


## Final Dataset

In [16]:
# Save the final DataFrame to a CSV
videos_df.to_csv('youtube_analysis_final.csv', index=False)

print("Final dataset 'youtube_analysis_final.csv' has been saved!")

Final dataset 'youtube_analysis_final.csv' has been saved!
